In [2]:
import numpy as np
import numpy as np
import pandas as pd
#import WindFarmGenetic_v1 # wind farm layout optimization using genetic algorithms classes
from datetime import datetime
import os
from sklearn.svm import SVR
import pickle


In [3]:
import os
os.getcwd()

'/home/naveen/arena/hackathon/shell/hackathon/python/Ver2_N'

In [4]:
##
wt_N=50  #number of turbines
NA_loc=[]  #index of obstructed locations
population_size=100  #population size/number of layouts for genetic algorithms
n_mc_samples=10000 #population/layout for SVM training
iteration_times=100  #number of iterations

# wind farm size, cells
cols_cells = 80  # number of cells each row
rows_cells = 80  # number of cells each column
cell_width = 50 # unit (TBD)--> have to be computed.
constraint_dist = 400 #unit distance between two turbines
constraint_perim = 50

radius=np.ceil(constraint_dist/cell_width)

#genetic algorithmic parameters.
elite_rate = 0.2 
cross_rate = 0.6
mutate_rate = 0.1

random_rate = 0.5
num_runs=2 #number of independent runs with random initialization.


shell_default_location='../../data/Shell_Hackathon_Dataset/'
wind_inst_freq_file=None

In [5]:
perim_cells=int(np.ceil(constraint_perim/cell_width)) #TBU generalize it for multiple boundary cells.
NA_loc=[] 

for curr_perim_cell in range(perim_cells):
    NA_loc.extend([i+cols_cells*curr_perim_cell for i in range(cols_cells)]) #upper row
    NA_loc.extend([i*(cols_cells)+curr_perim_cell for i in range(rows_cells)]) #left boundary
    NA_loc.extend([i*(cols_cells)+cols_cells-1-curr_perim_cell for i in range(rows_cells)])#right Boundary
    NA_loc.extend([(cols_cells)*(rows_cells-1-curr_perim_cell)+i for i in range(rows_cells)])#down Boundary


In [ ]:
paraDict = {}
paraDict['wt_N']=wt_N  #number of turbines
paraDict['population_size']=population_size  #population size/number of layouts for genetic algorithms
paraDict['n_mc_samples']=n_mc_samples #population/layout for SVM training
paraDict['iteration_times']=iteration_times  #number of iterations

# wind farm size, cells
paraDict['cols_cells'] = cols_cells  # number of cells each row
paraDict['rows_cells'] = rows_cells  # number of cells each column
paraDict['cell_width'] = cell_width # unit (TBD)--> have to be computed.
paraDict['constraint_dist'] = constraint_dist #unit distance between two turbines
paraDict['constraint_perim'] = constraint_perim

#genetic algorithmic parameters.
paraDict['elite_rate'] =elite_rate
paraDict['cross_rate'] = cross_rate
paraDict['mutate_rate'] = mutate_rate

paraDict['random_rate'] = random_rate
paraDict['num_runs']=num_runs #number of independent runs with random initialization.

In [ ]:
data_folder = "naveen_data" #temporary code related files. (MOdify it as per your specification)
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

wds_data_folder = "{}/wds".format(data_folder)
if not os.path.exists(wds_data_folder):
    os.makedirs(wds_data_folder)
    
# initial populations saved folder
init_pops_data_folder = "{}/init_data".format(data_folder)
if not os.path.exists(init_pops_data_folder):
    os.makedirs(init_pops_data_folder)

#result folder
results_data_folder = data_folder+"/results"
if not os.path.exists(results_data_folder):
    os.makedirs(results_data_folder)
    
#result folder for sugga genetic algorithm
sg_result_folder = "{}/sg".format(results_data_folder)
if not os.path.exists(sg_result_folder):
    os.makedirs(sg_result_folder)
fname= data_folder+"/"+"exp_specs.txt"
file = open(fname, 'w')
file.write(str(paraDict))
file.close()

In [7]:
import WindFarmGenetic_v2 as WindFarmGenetic
wfg = WindFarmGenetic.WindFarmGenetic(rows=rows_cells, cols=cols_cells, N=wt_N, NA_loc=NA_loc, pop_size=population_size,
                                      iteration=iteration_times,cell_width=cell_width, elite_rate=elite_rate,
                                             cross_rate=cross_rate, random_rate=random_rate, mutate_rate=mutate_rate,constraint_dist=constraint_dist,
                                     shell_default_location=shell_default_location, wind_inst_freq_file=wind_inst_freq_file)

Initialization Completed


In [8]:
for i in range(num_runs):
    wfg.gen_init_pop_NA()
    wfg.save_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))

In [9]:
##### SUGGA: support vector regression guided genetic algorithm
result_arr = np.zeros((num_runs, 2), dtype=np.float32)
for i in range(0, num_runs):  # run times
    print("run number: {} ...".format(i))
    wfg.load_init_pop_NA("{}/init_{}.dat".format(init_pops_data_folder, i),
                         "{}/init_{}_NA.dat".format(init_pops_data_folder, i))
    run_time, eta = wfg.sugga_genetic_alg(i, result_folder=sg_result_folder)
    result_arr[i, 0] = run_time
    result_arr[i, 1] = eta
time_stamp = datetime.now().strftime("%Y%m%d%H%M%S")
filename = "{}/result_sugga_{}.dat".format(sg_result_folder, time_stamp)
np.savetxt(filename, result_arr, fmt='%f', delimiter="  ")

run number: 0 ...


layouts:   0%|          | 0/100 [00:00<?, ?it/s]

Support vector regression guided genetic algorithm starts....
generation 0...


layouts: 100%|██████████| 100/100 [00:26<00:00,  3.71it/s]


[511.49484 511.28848 513.7745  514.4093  510.70505 513.7103  512.77026
 513.62585 513.0109  513.409   514.2308  509.19778 511.17148 512.0586
 511.9907  511.93945 510.4664  511.14325 510.46616 513.6012  514.0904
 511.66647 513.6516  511.814   514.1923  510.51117 513.1275  511.6015
 511.69775 513.20685 514.4995  511.2424  509.56165 513.32666 512.0605
 513.3083  510.9083  511.8038  510.67328 508.7553  511.88522 512.06146
 514.06866 513.6094  511.17813 513.5729  512.2916  512.7486  514.1534
 512.3881  513.6109  512.69904 510.99857 515.86053 512.51715 513.4411
 511.8924  512.43976 514.03827 513.3945  511.69766 511.306   513.00305
 513.06287 511.28497 513.2239  510.36768 512.10406 512.3381  511.8161
 511.8724  515.6726  509.00827 511.50003 511.5466  514.7827  511.15897
 513.44196 509.1693  510.78534 515.8305  512.3989  516.0541  510.18408
 512.9253  509.89398 511.05573 513.3103  511.7894  511.55634 512.16266
 513.74286 508.57144 514.5797  512.78046 512.52905 510.68896 510.13733
 510.79053 51

layouts:   1%|          | 1/100 [00:00<00:18,  5.25it/s]

generation 1...


layouts: 100%|██████████| 100/100 [00:23<00:00,  4.33it/s]


[516.2256  516.0216  516.6104  515.8379  514.9135  514.79614 515.9362
 513.4992  515.3092  515.14453 515.1035  514.92633 516.16016 514.89777
 514.6817  511.34082 515.08203 515.1487  512.8788  512.05524 514.3997
 514.48413 514.9368  511.48624 513.58234 514.33514 511.32394 512.0574
 513.02313 510.20853 514.37134 511.33966 512.5816  513.6611  511.56976
 513.2916  515.04877 511.99384 511.334   514.1963  514.64136 514.79956
 515.56915 514.43555 513.818   511.67578 511.76486 513.51587 513.1913
 513.5822  513.4627  512.26086 513.2528  512.3118  511.08096 515.0881
 514.60724 514.40466 512.3202  512.6053  513.5443  512.9233  514.12286
 511.31622 508.39145 513.33453 513.899   517.1191  514.2448  515.1074
 513.10077 512.3045  513.44476 511.9035  512.8408  511.46732 512.94446
 511.82977 511.5319  513.84503 515.2042  512.259   513.9773  513.4847
 515.2886  512.3876  514.3768  515.19977 513.6249  513.2005  511.05463
 515.10144 511.4986  511.40125 512.7806  515.26105 512.2434  512.4598
 516.08923 514

KeyboardInterrupt: 

### Shell Power Computation Package

In [1]:
import sys 
sys.path.append('../Shell_Code_Modified/')

In [2]:
import Farm_Evaluator_Vec

In [3]:
total_energy, turbine_power=Farm_Evaluator_Vec.get_AEP_results()

In [4]:
total_energy

array([1.2523587, 1.1595447, 1.1450489, 1.0768385, 1.176319 , 1.1840583,
       1.13568  , 1.079648 , 1.0935222, 1.1273198, 1.0859367, 1.167106 ,
       1.1016386, 1.124391 , 1.1225449, 1.100678 , 1.1647563, 1.2282459,
       1.1581823, 1.197542 , 1.2453799, 1.2442212, 1.2059841, 1.1259376,
       1.2380294, 1.1106503, 1.1325871, 1.1829029, 1.1153044, 1.1439447,
       1.1912581, 1.1264657, 1.1022447, 1.1789318, 1.1049944, 1.2037905,
       1.220672 , 1.1373823, 1.2131999, 1.1142861, 1.199062 , 1.141587 ,
       1.1335138, 1.1493688, 1.0722657, 1.0965657, 1.2305766, 1.2662541,
       1.1084578, 1.0826805], dtype=float32)

In [5]:
turbine_power

505.4506365966797